#### Imports

In [ ]:
from collections import Counter

import copy
import nltk
import pickle
import gensim
import multiprocessing
from itertools import combinations

import numpy as np
import pandas as pd

In [ ]:
import utils as my_utils

### Required Methods

In [ ]:
dataset = pd.read_pickle("datasets/datadf_amazon_musical")

In [ ]:
dataset.head(2)

In [ ]:
count_matrix, _, vocabulary, words = my_utils.processReviews(dataset['text'].values)

### Glove

In [ ]:
# def loadGloveModel(gloveFile):
#     print("Loading Glove Model")
#     f = open(gloveFile,'r')
#     model = {}
#     for line in f:
#         splitLine = line.split()
#         word = splitLine[0]
#         embedding = np.array([float(val) for val in splitLine[1:]])
#         model[word] = embedding
#     print("Done.",len(model)," words loaded!")
#     return model

In [ ]:
# %%time
# embeddings_index = loadGloveModel("nongit_resources/glove.6B.300d.txt")

In [ ]:
# words_embeddings = {}
# for i in words:
#     try:
#         words_embeddings[i] = embeddings_index[i]
#     except:
#         pass

### Fasttext

In [ ]:
# %%time
# embeddings_index = gensim.models.KeyedVectors.load_word2vec_format("nongit_resources/wiki-news-300d-1M.vec")

In [ ]:
# words_embeddings = {}
# for i in words:
#     try:
#         words_embeddings[i] = embeddings_index[i]
#     except:
#         pass

### Bert

In [ ]:
# from bert_embedding import BertEmbedding

In [ ]:
# bert_embedding = BertEmbedding()

In [ ]:
# result = bert_embedding(words)

In [ ]:
# words_embeddings = {}
# for i in result:
#     words_embeddings[i[0][0]] = i[1][0]

### Elmo

In [ ]:
from allennlp.commands.elmo import ElmoEmbedder

In [ ]:
elmo = ElmoEmbedder()

In [ ]:
result = elmo.embed_sentence(words) # taking only second layer

In [ ]:
result[2]

In [ ]:
words_embeddings = {}
for idx, i in enumerate(result[2]):
    words_embeddings[words[idx]] = i

### Embeddings

In [ ]:
embeddings_index = None

In [ ]:
len(words_embeddings)

In [ ]:
words_with_embeddings = words_embeddings.keys()

In [ ]:
%%time
edge_embeds_multi = []
for i, j in combinations(words_with_embeddings, 2):
    edge_embeds_multi.append((words_embeddings[i], words_embeddings[j]))

In [ ]:
len(edge_embeds_multi)

In [ ]:
n_cores = 35

In [ ]:
%%time
pool = multiprocessing.Pool(n_cores)
embeddings_cosines = pool.map(my_utils.get_cosine_multi, edge_embeds_multi)
pool.close()

In [ ]:
len(embeddings_cosines)

In [ ]:
%%time
edge_embeddings = {}
for idx, (i, j) in enumerate(combinations(words_with_embeddings, 2)):
    edge_embeddings[(i, j)] = embeddings_cosines[idx]
    edge_embeddings[(j, i)] = embeddings_cosines[idx]

In [ ]:
edges_threshold_all = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
def get_edges_per_doc(doc):
    edges = [[] for i in range(len(edges_threshold_all))]
    edges_sim = [[] for i in range(len(edges_threshold_all))]
    for i in doc:
        for j in doc:
            if i != j and i in words_with_embeddings and j in words_with_embeddings:
                sim = edge_embeddings[(i, j)]
                for idx, edges_threshold in enumerate(edges_threshold_all):
                    if sim > edges_threshold and (vocabulary[i], vocabulary[j]) not in edges[idx] and (vocabulary[j], vocabulary[i]) not in edges[idx]:
                        edges[idx] += [(vocabulary[i], vocabulary[j])]
                        edges_sim[idx] += [(i, j, sim)]
    return edges, edges_sim

In [ ]:
n_cores

In [ ]:
%%time
pool = multiprocessing.Pool(n_cores)
docs_edges_multi = pool.map(get_edges_per_doc, dataset['cleaned'].values)
pool.close()

In [ ]:
docs_edges_multi

In [ ]:
pickle_out = open("docs_edges_multi.pickle","wb")
pickle.dump(docs_edges_multi, pickle_out)
pickle_out.close()

In [ ]:
docs_edges = [i[1] for i in docs_edges_multi]

In [ ]:
dataset['text'].values[1]

In [ ]:
docs_edges_all[1]

In [ ]:
len(docs_edges_all[1])

In [ ]:
np.median([len(i) for i in docs_edges])

In [ ]:
np.mean([len(i) for i in docs_edges])

In [ ]:
np.where(np.array([len(i) for i in docs_edges]) == 0)[0].shape

In [ ]:
Counter([len(i) for i in docs_edges])

In [ ]:
pickle_out = open("resources/amazon_musical_glove_nontrained_0.00.pickle","wb")
pickle.dump(docs_edges, pickle_out)
pickle_out.close()

# Appendix

In [ ]:
# %%time
# edges_threshold = 0.73
# docs_edges = []
# docs_edges_all = []
# for idx, doc in enumerate(dataset[10].values):
#     my_utils.print_if_mod(idx, 500)
#     edges, edges_all = [], []
#     for i in doc:
#         for j in doc:
#             if i != j:
#                 sim = edge_embeddings[(i, j)]
#                 if sim > edges_threshold and (vocabulary[i], vocabulary[j]) not in edges and (vocabulary[j], vocabulary[i]) not in edges:
#                     edges.append((vocabulary[i], vocabulary[j]))
#                     edges_all.append((i, j, sim))
#     docs_edges.append(edges)
#     docs_edges_all.append(edges_all)

In [ ]:
# def loadGloveModel(gloveFile):
#     print("Loading Glove Model")
#     f = open(gloveFile,'r')
#     model = {}
#     for line in f:
#         splitLine = line.split()
#         word = splitLine[0]
#         embedding = np.array([float(val) for val in splitLine[1:]])
#         model[word] = embedding
#     print("Done.",len(model)," words loaded!")
#     return model

In [ ]:
# %%time
# embeddings_index = loadGloveModel("nongit_resources/glove.42B.300d.txt")

In [ ]:
# %%time
# embeddings_index = gensim.models.KeyedVectors.load_word2vec_format("nongit_resources/electronics.vec")

In [ ]:
# %%time
# edges_threshold = 0.8
# docs_edges, ignored, taken, count = [], [], [], 0
# for idx, doc in enumerate(dataset[8].values):
#     edges = []
#     print(idx)
#     for i in doc:
#         for j in doc:
#             if i != j:
#                 try:
#                     a = embeddings_index[i]
#                     b = embeddings_index[j]
#                     if get_cosine(a, b) > edges_threshold and (vocabulary[i], vocabulary[j]) not in edges and (vocabulary[j], vocabulary[i]) not in edges:
#                         edges.append((vocabulary[i], vocabulary[j]))
#                 except:
#                     try:
#                         embeddings_index[i]
#                         taken.append(i)
#                     except:
#                         ignored.append(i)
#                     try:
#                         embeddings_index[j]
#                     except:
#                         ignored.append(j)
#                         taken.append(j)
#                     pass
#     docs_edges.append(edges)

In [ ]:
# pickle_out = open("resources/docs_edges_" + dataset_name + "_5k_fasttext_trained.pickle","wb")
# pickle.dump(docs_edges, pickle_out)
# pickle_out.close()

In [ ]:
## Sentence wise
# dataset = dataset[['asin', 'helpful', 'overall', 'reviewText']]
# dataset['n_words'] = dataset['reviewText'].apply(lambda x: len(w_tokenizer.tokenize(x)))
# dataset['sentences'] = dataset['reviewText'].apply(lambda x: [i.strip() for i in x.split(".")])
# dataset['sentence_word_density'] = dataset['reviewText'].apply(lambda x: len(w_tokenizer.tokenize(x))/ len(x.split(".")))
# dataset.to_csv("reviews_Musical_Instruments_5.csv")

In [ ]:
# edge_dict, ignored, taken, count = {}, [], [], 0
# for idxi, i in enumerate(vocabulary.keys()):
#     print(idxi)
#     for idxj, j in enumerate(vocabulary.keys()):
#         if i != j:
#             try:
#                 a = embeddings_index[i]
#                 b = embeddings_index[j]
#                 if get_cosine(a, b) > edges_threshold:
#                     try:
#                         edge_dict[vocabulary[i]] += [vocabulary[j]]
#                         edge_dict[vocabulary[j]] += [vocabulary[i]]
#                     except:
#                         edge_dict[vocabulary[i]] = [vocabulary[j]]
#                         edge_dict[vocabulary[j]] = [vocabulary[i]]
#             except:
#                 try:
#                     embeddings_index[i]
#                     taken.append(i)
#                 except:
#                     ignored.append(i)
#                 try:
#                     embeddings_index[j]
#                 except:
#                     ignored.append(j)
#                     taken.append(j)
#                 pass

In [ ]:
# df = {}

# for idx, i in enumerate(dataset[8].values):
#     print(idx)
#     for j in i:
#         try:
#             df[j] += [idx]
#         except:
#             df[j] = [idx]

# for i in df.keys():
#     df[i] = len(list(set(df[i])))

# df_vector = []
# for i in dataset[8].values:
#     d = [0]*len(vocabulary.keys())
#     for j in i:
#         if j in vocabulary.keys():
#             d[vocabulary[j]] = df[j]
#     df_vector.append(d)

# csr = sparse.csr_matrix(np.array(df_vector))
# scipy.sparse.save_npz('resources/df_stackoverflow_5kanswers.npz', csr)

In [ ]:
# np.array(scipy.sparse.load_npz('resources/df_stackoverflow_5kanswers.npz').todense())

In [ ]:
# dataset = parse("nongit_resources/reviews_Electronics_5.json.gz")
# dataset = pd.DataFrame(list(dataset))
# dataset = dataset.head(N_docs)
# dataset.to_pickle("resources/reviews_Electronics_5")